In [12]:
from typing import Tuple
from types import SimpleNamespace
import os
import sys
from pathlib import Path
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from data_loaders.example_data_loader import ExampleDatasetGenerator
from models.models import JukeboxModel
from models.preprocessing import AudioFeatureExtractor
from utils.dirs import create_dirs
from utils.config import process_config
from utils.utils import get_args

import datasets.canonne_duos.canonne_duos

In [14]:
config_path = './configs/something_descriptors_config.json'
save_root_path = '/data/anasynth_nonbp/thiel/saved_models/listening_tests/'
dataset_path = '/data/anasynth_nonbp/thiel/tensorflow_datasets/'
# capture the config path from the run arguments
# then process the json configuration file
args = SimpleNamespace(
    config=config_path,
    save_root=save_root_path,
    dataset=dataset_path
)
config = process_config(args)

# create the experiments dirs
create_dirs([config.save.path.log_dir,
            config.save.path.checkpoint_dir])
print(config)

namespace(exp_name='Something', data=namespace(audio=namespace(time=namespace(size_win=1024, stride_win=256, rate_sample=48000), freq=namespace(freq_min=80, freq_max=10000, nb_mfcc=32), features=namespace(kind='descriptors', names=['freq_0', 'prob_freq', 'root_mean_square', 'zero_crossing_rate', 'spectral_flatness', 'spectral_centroid'])), annotations=namespace(size_kernel=3, sigma=1)), model=namespace(jukebox=namespace(nb_levels=2, nb_filters=8, nb_blocks_sample=[3, 5], size_kernel_sample=4, stride_sample=2, nb_blocks_res=[4, 2], size_kernel_res=3, rate_dilation_res=3, size_codebook=256, beta_codebook=0.99)), loss=namespace(), training=namespace(rate_learning=0.02, size_batch=8, nb_epochs=10), dataset=namespace(name='canonne_duos', validation_split=0.1, path=PosixPath('/data/anasynth_nonbp/thiel/tensorflow_datasets')), save=namespace(log=namespace(update_freq=1), checkpoint=namespace(max_to_keep=5, checkpoint_interval=2), path=namespace(log_dir='/data/anasynth_nonbp/thiel/saved_models

In [17]:
import manage_gpus as gpl
try:
    id_gpu_locked = gpl.get_gpu_lock(soft=True)
    comp_device = f"/gpu:{id_gpu_locked}"
    # os.environ["CUDA_VISIBLE_DEVICES"]=""
except gpl.NoGpuManager:
    print("no gpu manager available - will use all available GPUs", file=sys.stderr)
except gpl.NoGpuAvailable:
    # there is no GPU available for locking, continue with CPU
    comp_device = "/cpu:0" 
    os.environ["CUDA_VISIBLE_DEVICES"]=""

if "CUDA_VISIBLE_DEVICES" in os.environ and len(os.environ["CUDA_VISIBLE_DEVICES"]) > 0:
    devices = tf.config.list_physical_devices('GPU')
    print(len(devices))
    tf.config.experimental.set_memory_growth(device=devices[0], enable=True)
else:
    comp_device = "/cpu:0" 

no gpu manager available - will use all available GPUs


In [16]:
with tf.device(comp_device):

    ds_train, ds_val = tfds.load(
        'canonne_duos',
        split=['train[90%:]', 'train[:10%]'],
        data_dir=config.dataset.path,
        download=False
    )
    
    shape_preprocessed = ds_train.element_spec['audio'].shape

    feature_extractor = AudioFeatureExtractor.make(
        kind_feature=config.data.audio.features.kind,
        name_features=config.data.audio.features.names,
        rate_sample=config.data.audio.time.rate_sample,
        size_win=config.data.audio.time.size_win,
        stride_win=config.data.audio.time.stride_win,
        freq_min=config.data.audio.freq.freq_min,
        freq_max=config.data.audio.freq.freq_max,
        nb_mfcc=config.data.audio.freq.nb_mfcc
    )

    @tf.function
    def _preprocess(inputs: dict) -> Tuple[tf.Tensor, tf.Tensor]:
        x_audio = inputs['audio']
        x_features = feature_extractor(x_audio)
        x = x_features
        y = x
        return x, y
    
    # for now, process left and right channel separately
    # stereo example as two mono examples -> unbatch then batch again.
    ds_train = ds_train.map(_preprocess) \
        .unbatch() \
        .shuffle(4) \
        .batch(config.training.size_batch)

ParseError: Failed to construct dataset canonne_duos: Message type "tensorflow_datasets.DatasetInfo" has no field named "fileFormat" at "DatasetInfo".
 Available Fields(except extensions): "['name', 'description', 'version', 'configName', 'configDescription', 'citation', 'sizeInBytes', 'downloadSize', 'location', 'downloadChecksums', 'schema', 'splits', 'supervisedKeys', 'redistributionInfo', 'moduleName', 'disableShuffling']"